In [1]:
%load_ext jupyter_black

In [44]:
import os
import pathlib
import instructor
from pydantic import BaseModel
from openai import OpenAI
import getpass
import pymupdf
import pymupdf4llm


In [7]:
_API_KEY = getpass.getpass("Your API key: \n")

Your API key: 
 ········


In [8]:
open_ai = OpenAI(api_key=_API_KEY)
client = instructor.from_openai(open_ai, mode=instructor.Mode.TOOLS_STRICT)

In [9]:
class Response(BaseModel):
    message: str

In [13]:
def get_response(query, url):
    return client.chat.completions.create(
        model="gpt-4o-mini",
        response_model=Response,
        messages=[
            {
                "role": "developer",
                "content": "You are a world-class expert. You will analyze the document at the given URL and answer the query.",
            },
            {"role": "user", "content": f"url: {url}"},
            {
                "role": "user",
                "content": f"{query}",
            },
        ],
    )

In [14]:
r = get_response(
    "Who is the highest paid employee?",
    "https://budgetandfinance.psu.edu/sites/budgetandfinance/files/right_to_know_2023_0.pdf",
)

In [15]:
print(r.message)

I currently cannot access external websites or documents directly. However, you can view the document at the provided URL and search for sections that typically contain such information, like "Highest Paid Employees" or similar headings. If you need further assistance with analyzing the document after you've accessed it, feel free to ask!


In [36]:
# data from https://budgetandfinance.psu.edu/budget-allocations

table = """
  UP Colleges                              FY26 Budget   FY27 Budget   FY27 Reduction   FY27 Change
  -------------------------------------- ------------- ------------- ---------------- -------------
  Agriculture Sciences (General Funds)      32,230,886    32,762,270          531,384         1.65%
  Arts & Architecture                       42,370,572    42,785,632          415,060         0.98%
  Business, Smeal                           88,192,376    94,641,413        6,449,037         7.31%
  Communications, Bellisario                22,147,211    25,462,815        3,315,604        14.97%
  Earth & Mineral Sciences                  51,423,487    51,643,168          219,680         0.43%
  Education                                 37,565,599    36,229,442      (1,336,157)        -3.56%
  Engineering                              128,971,069   129,244,355          273,286         0.21%
  Health & Human Development                67,487,969    72,077,693        4,589,724         6.80%
  Information Sciences & Technology         32,587,758    36,627,971        4,040,213        12.40%
  International Affairs                      2,464,550     2,054,674        (409,876)       -16.63%
  Liberal Arts                             164,960,545   170,149,349        5,188,805         3.15%
  Nursing, Nese                             17,071,041    16,710,190        (360,850)        -2.11%
  Penn State Dickinson Law                  19,023,564    16,725,000      (2,298,564)       -12.08%
  Science, Eberly                          128,190,811   134,408,900        6,218,089         4.85%
"""



In [37]:
class BudgetAnswer(BaseModel):
    amount: float

In [59]:
def extract_from_table(query, table):
    return client.chat.completions.create(
        model="gpt-4o-mini",
        response_model=BudgetAnswer,
        messages=[
            {
                "role": "developer",
                "content": "You are a world-class expert. You will analyze the given markdown tables and answer the query.",
            },
            {"role": "user", "content": f"tables: {table}"},
            {
                "role": "user",
                "content": f"query: {query}",
            },
        ],
    )

In [60]:
q = "How much money is allocated for Information Sciences & Technology in FY27?"
r = extract_from_table(q, table)

In [42]:
# If you want to pretty print: 
# print(f"{r.amount:0,.2f}")
# 36,627,971.00

In [61]:
print(r.amount)

36627971.0


# Todo: Which UP colleges have the highest positive and negative percentage changes?

# Todo: Analyze the allocation for other units


Steps:


1. Add data from: `other-units-allocation.md`
2. Come up with 3 questions to answer
3. Complete code to answer these questions
4. Check answers for accuracy

# Using data from pdf

I'd like to use data from [Right To Know Law Reports from Penn State](https://budgetandfinance.psu.edu/public-reports#:~:text=Right%20To%20Know%20Law%20Report).

# What if we try to extract data directly from the pdf file?

Two steps:
* extract textual data from the pdf using an external library `pymupdf4llm`
* use the textual data as part of the prompt

First, we will have to make the pdf file available. If you are using the class `jupyterhub`,
you will need to the following two steps:

1. Create two folders: `data` and within it `data-activity-03` using the interface. That is,
the folder structure should be nested (`data/data-activity-03`)

2. Then, upload the `psu-right-to-know-form-990.pdf` file to `data-activity-03`

In [56]:
form_990_2023_md = pymupdf4llm.to_markdown("./data/data-activity-03/2023-psu-right-to-know-form-990.pdf")

Processing ./data/data-activity-03/2023-psu-right-to-know-form-990.pdf...
]========================================] (6/6)


In [57]:
# how does it look?
# first 1000 hundred chars
print(form_990_2023_md[:1000])

# Part VII Compensation of Officers, Directors, Trustees, Key Employees, Highest Compensated Employees, and Independent Contractors Check if Schedule O contains a response or note to any line in this Part VII . . . . . . . . . . . . . Section A. Officers, Directors, Trustees, Key Employees, and Highest Compensated Employees
**1a** Complete this table for all persons required to be listed. Report compensation for the calendar year ending with or within the
organization’s tax year.

  - List all of the organization’s current officers, directors, trustees (whether individuals or organizations), regardless of amount of
compensation. Enter -0- in columns (D), (E), and (F) if no compensation was paid.

  - List all of the organization’s current key employees, if any. See the instructions for definition of “key employee.”

  - List the organization’s five **current** highest compensated employees (other than an officer, director, trustee, or key employee)
who received reportable compensation 

In [51]:
def extract_from_markdown(query, markdown_source):
    return client.chat.completions.create(
        model="gpt-4o-mini",
        response_model=Response,
        messages=[
            {
                "role": "developer",
                "content": "You are a world-class expert. You will analyze the given markdown content and answer the query.",
            },
            {"role": "user", "content": f"markdown: {markdown_source}"},
            {
                "role": "user",
                "content": f"query: {query}",
            },
        ],
    )

In [58]:
q = "How much did James Franklin earn?"
r = extract_from_markdown(q, form_990_2023_md)

In [53]:
print(r.message)

James Franklin earned a total reportable compensation of **$8,455,055** from the organization for the calendar year.


# ToDo: Could we use BudgetAnswer class as `response_model` here?

# ToDos:

* How much did the CEO of Penn State Health earn?
* What was the biggest source of PSU revenue?
* How much did Penn State spend on advertising and promotion?
* How much did IST Dean earn?

# ToDo: Analyze the 2018 Form 990

## Following the 2018 form:

* Who was the top earner at Penn State?
* Who was the top fifth earner at Penn State?
* How much did the CEO of Penn State Health earn?
* What was the biggest source of PSU revenue?
* How much did Penn State spend on advertising and promotion?
* How much did IST Dean earn?